#### MAE 3120
## HW 05

Due 04/18

For full credit, show all your work.  

### 1  Strain gage optimization

A force balance can be modeled as a horizontal canteliver beam: one end is clamped and the other is free. The beam has a rectangular cross-section (width $b$ and height $h$).  The expected stress from a downward force applied at fixed point on the beam is: 

\begin{align}
\sigma_a = 6 \frac{F x }{b h^2}
\end{align}
Where, $F$ is the force applied (here a 1 kg weight) and $x$ is the distance from center of force to the clamped edge of the beam.  Overall the beam has a cantilever length $l=100.0\pm0.1\,\text{mm}$.

Here: <br>
$h=4.00\pm0.05 \,\text{mm}$ <br>
$b=12.00\pm0.05 \,\text{mm}$ <br>
$E = 207$ GPa, Young's modulus of the beam <br>

One strain gauge is mounted on the top of the beam the weight.  Its strain gauge factor is $S=2.00$. It is mounted on a wheatstone bridge 

1- To optimize force the weight measurement where will you position along the beam length, the strain gauge and the weight?  Justify your answer.

To get the best results, you want to maximize the sensitivity of the strain measurment, in other words the amount of strain, and therefore maximize the axial stress. 

To maximize stress, you should place the weight at the end of the beam (maximize x). 

The stress is constant along the beam, so the strain gauge can be located anywhere along the beam.

2- The load cell is located at $x_0=10.00\pm0.04\,\text{mm}$ and the weight is anchored at $x_1=95.00\pm0.1\,\text{mm}$.  Estimate the stress and resulting strain at the strain gauge location.

In [48]:
import numpy as np

F = 1 * 9.81 #N
x_o = 0.01 #m
x_1 = 0.095 #m
h = 0.004 #m
b = 0.012 #m
l = 0.1 #m
E = 207E9 #Pa
S = 2

sigma = 6*F*x_1/(b*h**2)

eps = sigma/E

print('the stress is sigma = ', sigma*10**-6, 'MPa')
print('the strain is epsilon_a = ', eps*10**6, 'microstrain')

the stress is sigma =  29.1234375 MPa
the strain is epsilon_a =  140.6929347826087 microstrain


3- Estimate the uncertainty in the weight measurement from the beam geometry and strain gauge alone. 

Overall the strain is:
\begin{align*}
\epsilon_a = 6E\frac{F x_1}{bh^2}
\end{align*}
or the force is related to the strain by:
\begin{align*}
F = \frac{b h^2 E}{6 x_1} \epsilon_a
\end{align*}


One can recognize that this is the uncertainty of a result.  Since the equation is polynomial, we can use the simplified form of the relative uncertainty:

\begin{align*}
\frac{u_R}{R} = \sqrt{\sum_i \left( a_i \frac{u_{x_i}}{x_i} \right)^2}
\end{align*}

Which here reduces to:
\begin{align*}
\frac{u_F}{F} = \sqrt{\left( 1 \frac{u_b}{b} \right)^2 + \left( 2 \frac{u_h}{h} \right)^2 + \left( 1 \frac{u_{\epsilon_a}}{\epsilon_a} \right)^2 + \left( -1 \frac{u_{x_1}}{x_1} \right)^2} 
\end{align*}
with 
\begin{align*}
\frac{u_b}{b} & = 0.05/12=1.25\, \%\\
\frac{u_h}{h} & = 0.05/4=0.42\, \% \\ 
\frac{u_{x_1}}{x_1} & = 0.1/95=0.11\,\%
\end{align*}
with $u_E$ negligible and $u_{\epsilon_a}$ that will be determined from the analysis of the DAQ.

In [49]:
u_h = 0.05/4 #%
u_b = 0.05/12 #%
u_x1 = 0.1/95 #%

u_FF = np.sqrt((1*u_b)**2 + (2*u_h)**2 + (-1*u_x1)**2)

print('u_h=',u_h, 'u_b=',u_b, 'u_x1=', u_x1)
print('uncertainty in the force due to beam geometry u_F/F', u_FF*100,'%')

u_h= 0.0125 u_b= 0.004166666666666667 u_x1= 0.0010526315789473684
uncertainty in the force due to beam geometry u_F/F 2.5366693603071497 %


4- What should be the number of bit of the DAQ system to have a negligible effect on your force measurement accuracy? Assume $V_s = 10\,\text{V}$ and DAQ range is $\pm 100 \,\text{mV}$. Justify your answer.

We are using a quarter bridge. The voltage output will be:
\begin{align*}
V_o = \frac{1}{4} \epsilon_a S V_s
\end{align*}
Overall, we do not want the DAQ to contribute significantly to the overall uncertainty of 2.5%, i.e we aim for no more than 10% increase in uncertainty.  So we will be aiming for a quantization error $Q$ of 1% of the output voltage $V_o$, ie the $u_{{\epsilon_a}}$.  Remember the definition of $Q$.
\begin{align*}
Q = \frac{V_{max}-V_{min}}{2^{N+1}}
\end{align*}
Here $V_{max}-V_{min}=0.2$ V.

In [50]:
V_s = 10 # V
S = 2.00
V_o = eps*S*V_s/4
print('V_o = ',V_o*1000,'mV')

N=14 # # of bits
Vrange = 0.2
Q = Vrange/(2**(N+1))
print('Q =',Q*1000,'mV')

u_eps = Q/V_o
print('u_epsilon = ',u_eps*100,'%')
# We check that the overall uncertainty is not increased significantly
u_FF2 = np.sqrt(u_FF**2+u_eps**2)
# change in uncertainty
deltaU_FF = (u_FF2-u_FF)/u_FF
print('u_FF2 = ',u_FF2,'%')
print('change in uncertainty is ',deltaU_FF*100,'%')
print('We would need a DAC with at least N =',N,'bit.')

V_o =  0.7034646739130436 mV
Q = 0.006103515625 mV
u_epsilon =  0.867636407532593 %
u_FF2 =  0.026809484103945656 %
change in uncertainty is  5.687735750864507 %
We would need a DAC with at least N = 14 bit.


### 2 Strain Gage measurement system

A quarter-bridge Wheatstone bridge circuit is used with a strain gage to measure strain up to $1000\, \mu$strain for a cylindrical, vertical cable undergoing cyclical loading at $0.1$ Hz. 

In addition, you know that:

_ There is noise on the input: $f_{noise}=60$ Hz with amplitude $\pm 0.050$ mV. </br>
_ The output is sent to a DAQ board, which has a 12 bit A/D converter with a controllable range of $\pm 0.1, 1, 5, 10$ V.</br>
_ All Wheatstone bridge resistors have resistance $120\,\Omega$ including the unloaded strain gage, resulting in a balanced bridge at zero strain.</br>
_ The strain factor is $S=2.00$.</br>
_ The supply voltage for the Wheatstone bridge is $V_s=5.00$ VDC.


(a) How would you mount the strain gage on the cable and on your wheatstone bridge to get a positive voltage for a positive strain?  Justify your steps.

For a positive stress, the vertical cables will be elongated and therefore will experience a positive strain.  The strain gage will be mounted along the axis of the cable.  It should be mounted either on branches $R_1$ or $R_3$ of the Wheatstone bridge.

(b) What would be the maximum output voltage you expect to measure?

We are using a quarter bridge. The voltage output will be:
\begin{align*}
V_o = \frac{1}{4} \epsilon_a S V_s
\end{align*}

In [51]:
V_s = 5 # V
eps_max = 1000E-6 # strain
S = 2.00 # Strain gage factor
V_o = eps_max*S*V_s/4
print('the output voltage is V_o =',V_o*1000,'mV')

the output voltage is V_o = 2.5 mV


(c) Which range would you select on your DAQ?  

I would choose the smallest range of $\pm0.1$ V.

(d) You will need to amplify your signal to make better use of the range of your DAQ.  Select (justifying why) the value of the gain.  How many amplifier stages are necessary.

To make the best use of the DAQ while preventing clipping, one would need to amplify the signal by $G_{amp}=30$.  We need to check that this gain does not create impedance loading issues in our signal conditioning and respect the $GBP$ of the Op-Amp.

- Impedance loading.

$G_{amp}$ is 30, which is the upper acceptable limit.

- $GBP < 1$ MHz

The Bandwidth of interest is $B = 0.1$ Hz.  We are satisfying this condition too.

In conclusion, a single amplifier stage of $G_{amp}=30$ is adequate for amplifying the signal from the Wheatstone bridge.

(e) How would you filter the noise in the amplified data?  I.e. give the value of the cutoff frequency, order of the filter.  Justify all your steps.

We need a low-pass filter to remove the 60 Hz noise.  Remember for low-pass filter of order $n$, the gain, $G_{LPF}$, and phase, $\phi_{LPF}$, are:

\begin{align*}
G_{LPF} & = \frac{1}{\sqrt{1+ \left( \frac{f}{f_{cutoff}}\right)^{2n} }} \\
\phi_{LPF} & = -\arctan \left( \frac{f}{f_{cutoff}} \right)
\end{align*}

We aim for a cuttoff frequency, $f_{cutoff}$, far enough from the carrier frequency, $f_{carrier}=0.1$ Hz, and the noise harmonic, $f_{noise}=60$ Hz: 

\begin{align}
10\times f_{carrier} = 1\,\text{Hz}<f_{cutoff}<6\,\text{Hz} = f_{noise}/10
\end{align}

I choose $f_{cutoff}=2$ Hz as an initial value.

Remember that the noise has also been amplified by the amplifier stage and its amplified amplitude is $A_{noise}=G_{amp}\times 5\times 10^{-5}$.

Here we need to make sure that the filtered noise does not register on the DAQ, ie its filtered amplitude should be smaller than the quantization error, $Q=V_{range}/2^{(N+1)}$ of the DAQ and double check that the carrier frequency has not been distorted significantly by checking that its phase shift is less than $|10^\circ|$.

In [52]:
G_amp = 30
A_noise = G_amp * 5E-5 # V
f_cutoff = 1 # Hz
f_carrier = 0.1 # Hz
f_noise = 60 # Hz
n = 1 # LPF order
N = 12 # ADC # bit
Q = 0.2/2**(N+1) # ADC quantization error
print('Q =',Q*1000,'mV')

A_noiseLPF = A_noise/np.sqrt(1+(f_noise/f_cutoff)**(2*n))
print('A_noiseLPF = ', A_noiseLPF*1000,'mV')

Q = 0.0244140625 mV
A_noiseLPF =  0.02499652850099 mV


Note, I had to decrease $f_{cutoff}$ to 1 Hz to have an acceptable design with a filter of order 1.  I now need to check the phase distortion.

In [53]:
phi = -np.arctan(f_carrier/f_cutoff) # phase in radian
print('phi =',phi*180/np.pi,'deg')

phi = -5.710593137499643 deg


This distortion is acceptable.

In summary, I can use a first order low-pass filter with cutoff frequency $f_{cutoff} = 1$ Hz.  

(f) Propose a new configuration that makes use of two strain gages to increase the sensitivity.  

We could add a second strain gage and create a 1/2 bridge by having a strain gage on $R_1$ and the other on $R_3$.

(g) Could you use a full bridge in this configuration?  Justify why.

No, all strain gages see a positive strain (the rod is in tension), and one cannot implement a full bridge.

### 3  Strain gages

Two identical 120 $\Omega$ strain gages are mounted $90^\circ$ to each other (in the $x$ and $y$ directions) on the metal surface of a beam under no-load conditions. The material properties are $E = 95.3$ GPa and $\nu = 0.305$. It is known that $x$ and $y$ are principal axes. Each strain gage is connected to a separate, but identical, quarter-bridge Wheatstone bridge with $V_s = 6.00$ VDC. Resistor $R_3$ is used as the strain gage in each of the two bridge circuits. The strain gage factor is $S = 2.081$ for both strain gages, and the bridges are balanced when there is no load. 

When a load is applied to the surface (loading in both the $x$- and $y$-directions), the output voltage of the bridge associated with the $x$-direction strain gage reads $V_{o,x} = 3.16$ mV, and that associated with the $y$-direction strain gage reads $V_{o,y} = -2.03$ mV.

(a) Calculate $\epsilon_x$ and $\epsilon_y$ in units of microstrain, and calculate $\sigma_x$ and $\sigma_y$ in units MPa.

We are using a quarter bridge with the strain gages mounted on $R_3$. The voltage output will be:
\begin{align*}
V_o & = \frac{1}{4} \epsilon_a S V_s \\
\epsilon_a & = \frac{4 V_o}{S V_s}
\end{align*}
The stress is related to the strain by the Young's modulus $E$.
\begin{align}
\sigma = E \epsilon
\end{align}

In [54]:
V_s = 6 # V
S = 2.081
E = 95.3E9 # Pa
V_ox = 3.16E-3 # V
V_oy = -2.03E-3 # V

epsilon_x = 4*V_ox/S/V_s
epsilon_y = 4*V_oy/S/V_s
print('epsilon_x = ',epsilon_x*1E6,'micro-strain')
print('epsilon_y = ',epsilon_y*1E6,'micro-strain')

print('sigma_x = ', epsilon_x*E*1E-6,'MPa')
print('sigma_y = ', epsilon_y*E*1E-6,'MPa')

epsilon_x =  1012.3338138715361 micro-strain
epsilon_y =  -650.3283677719046 micro-strain
sigma_x =  96.47541246195739 MPa
sigma_y =  -61.9762934486625 MPa


(b) Now you want to mount the two strain gages on a single 1/2 Wheatstone bridge.  Which resistors will you use to have the maximum sensitivity?  What would be the output voltage.

I would mount the strain gage that measures $\epsilon_x$ to non-inverting leg of the bridge ($R_1$ or $R_3$), and the other strain gage to an inverting leg ($R_2$ or $R_4$).

The output voltage will be:
\begin{align}
V_o = 5.19\,\text{mV}
\end{align}